<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Web-Scraping-using-BeautifulSoup" data-toc-modified-id="Web-Scraping-using-BeautifulSoup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Web Scraping using BeautifulSoup</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Cleaning</a></span><ul class="toc-item"><li><span><a href="#Adding-features-for-Stars" data-toc-modified-id="Adding-features-for-Stars-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Adding features for Stars</a></span></li></ul></li><li><span><a href="#Adding-New-Source-for-bankable-index" data-toc-modified-id="Adding-New-Source-for-bankable-index-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Adding New Source for bankable index</a></span><ul class="toc-item"><li><span><a href="#To-remove" data-toc-modified-id="To-remove-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>To remove</a></span></li></ul></li></ul></div>

In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from itertools import chain
import time
import sys
import numpy as np
import pandas as pd
import re

# Web Scraping using BeautifulSoup

In [2]:
import pandas
#blocks= ["1","51","101","151","201","251","301","351","401","451","501","551","601","651","701","751","801","851","801","851","901","951"]
blocks= np.arange(1,1100,50)
movie_first_line= []
movie_date = []
movie_run_time=[]
movie_genre =[]
cast=[]
movie_director =[]
movie_stars = []
gross_amount = []


for block in blocks:
    url= "https://www.imdb.com/search/title/?year=2018-01-01,2018-12-31&start={}".format(block)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "html")
    content = soup.find(id="main")

    movieFrame = content.find_all("div", class_="lister-item mode-advanced")
    for container in movieFrame:
#Movie first line
        movieFirstLine = container.find("h3", class_="lister-item-header")   
        movieTitle = movieFirstLine.find("a").text
        try:
            movie_first_line.append(movieTitle)
        except:
            movie_first_line.append(np.nan)
#Movie Date
        try:
            movieDate = re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text)
            movie_date.append(movieDate)
        except:
            movie_date.append(np.nan)

#Movie runtime
        try:
            movieRunTime = container.find("span", class_="runtime").text[:-4]
            movie_run_time.append(movieRunTime)
        except:
            movie_run_time.append(np.nan)
                
# Movie Genre
        try:
            movieGenre = container.find("span", class_="genre").text.rstrip().replace("\n","").split(",")
            movie_genre.append(movieGenre)
        except:
            movie_genre.append(np.nan)
                
#Casts         
        try:
            movieCast = container.find_all("p", class_="")[-1].text
            casts = movieCast.replace("\n","").split('|')
            casts = [x.strip() for x in casts]
            casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
# # Movie Director
            movie_director.append(casts[0])
# # # Movie Stars
            #movie_stars.append([x.strip() for x in casts[1].split(",")])
            movie_stars.append(casts[1])
        except:
            #casts = movieCast.text.replace("\n","").strip()
            movie_director.append(np.nan)
            #movie_stars.append([x.strip() for x in casts[1].split(",")])
            movie_stars.append(np.nan)
# Movie Gross
        try:
            gross_ind = container.find_all('span',attrs={"name":"nv"})[-1].text
            #gross_amount.append(gross_ind)
            if "M" not in gross_ind:
                mill = gross_ind.replace(",","")
                gross_ind = "0."+mill[:2]
            else:
                gross_ind= gross_ind.replace("M","")
                gross_ind = gross_ind.replace("$","")
            gross_amount.append(gross_ind)
            
    
        except:
            gross_amount.append(np.nan)
movie_data = [movie_first_line,movie_date,movie_run_time,movie_genre,movie_director,movie_stars,gross_amount]
output_df= pd.DataFrame(movie_data)


In [3]:
df= output_df

In [4]:
df= df.transpose()

In [5]:
df.head()

,0,1,2,3,4,5,6
0,Bohemian Rhapsody,2018,134,"[Biography, Drama, Music]",Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy",216.43
1,Spider-Man: Into the Spider-Verse,2018,117,"[Animation, Action, Adventure]","Directors:Bob Persichetti, Peter Ramsey, Rodne...","Shameik Moore, Jake Johnson, Hailee Steinfeld,...",190.24
2,Yellowstone,2018–,60,"[Drama, Western]",NaN,NaN,0.10
3,Hereditary,2018,127,"[Drama, Horror, Mystery]",Ari Aster,"Toni Collette, Milly Shapiro, Gabriel Byrne, A...",44.07
4,Killing Eve,2018–,43,"[Drama, Thriller]",NaN,NaN,0.48


# Data Cleaning

In [6]:
df.shape

(1100, 7)

In [7]:
# Renaming columns
df = df.rename(columns={0: "Movie_Name", 1: "Year",2:"Duration",3:"Genres",4:"Directors",5:"Stars", 6: "Gross"})




In [8]:
df.head()

,Movie_Name,Year,Duration,Genres,Directors,Stars,Gross
0,Bohemian Rhapsody,2018,134,"[Biography, Drama, Music]",Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy",216.43
1,Spider-Man: Into the Spider-Verse,2018,117,"[Animation, Action, Adventure]","Directors:Bob Persichetti, Peter Ramsey, Rodne...","Shameik Moore, Jake Johnson, Hailee Steinfeld,...",190.24
2,Yellowstone,2018–,60,"[Drama, Western]",NaN,NaN,0.10
3,Hereditary,2018,127,"[Drama, Horror, Mystery]",Ari Aster,"Toni Collette, Milly Shapiro, Gabriel Byrne, A...",44.07
4,Killing Eve,2018–,43,"[Drama, Thriller]",NaN,NaN,0.48


In [9]:
# Dropping any line with a na in it
df= df.dropna(how ='any')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796 entries, 0 to 1099
Data columns (total 7 columns):
Movie_Name    796 non-null object
Year          796 non-null object
Duration      796 non-null object
Genres        796 non-null object
Directors     796 non-null object
Stars         796 non-null object
Gross         796 non-null object
dtypes: object(7)
memory usage: 49.8+ KB


In [11]:
#Dropping the year column, they all contain 2018 and there is no adding value of keeping it
df["Year"].value_counts()

2018             650
I 2018            64
2018 TV Movie     39
II 2018           21
2018 Video        10
III 2018           9
V 2018             2
IX 2018            1
Name: Year, dtype: int64

In [12]:
del[df["Year"]]

In [13]:
# Removing white space from the Directors
df["Directors"] = df["Directors"].str.replace("Directors:","")

In [14]:
df.head()

,Movie_Name,Duration,Genres,Directors,Stars,Gross
0,Bohemian Rhapsody,134,"[Biography, Drama, Music]",Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy",216.43
1,Spider-Man: Into the Spider-Verse,117,"[Animation, Action, Adventure]","Bob Persichetti, Peter Ramsey, Rodney Rothman","Shameik Moore, Jake Johnson, Hailee Steinfeld,...",190.24
3,Hereditary,127,"[Drama, Horror, Mystery]",Ari Aster,"Toni Collette, Milly Shapiro, Gabriel Byrne, A...",44.07
5,Mary Poppins Returns,130,"[Comedy, Family, Fantasy]",Rob Marshall,"Emily Blunt, Lin-Manuel Miranda, Ben Whishaw, ...",171.96
6,A Star Is Born,136,"[Drama, Music, Romance]",Bradley Cooper,"Lady Gaga, Bradley Cooper, Sam Elliott, Greg G...",215.29


In [15]:
df.drop_duplicates(subset ="Movie_Name", 
                     keep = 'first', inplace = True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 794 entries, 0 to 1099
Data columns (total 6 columns):
Movie_Name    794 non-null object
Duration      794 non-null object
Genres        794 non-null object
Directors     794 non-null object
Stars         794 non-null object
Gross         794 non-null object
dtypes: object(6)
memory usage: 43.4+ KB


In [17]:
df= df[["Movie_Name","Genres","Duration","Gross","Directors","Stars"]]
df.head()

,Movie_Name,Genres,Duration,Gross,Directors,Stars
0,Bohemian Rhapsody,"[Biography, Drama, Music]",134,216.43,Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy"
1,Spider-Man: Into the Spider-Verse,"[Animation, Action, Adventure]",117,190.24,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Shameik Moore, Jake Johnson, Hailee Steinfeld,..."
3,Hereditary,"[Drama, Horror, Mystery]",127,44.07,Ari Aster,"Toni Collette, Milly Shapiro, Gabriel Byrne, A..."
5,Mary Poppins Returns,"[Comedy, Family, Fantasy]",130,171.96,Rob Marshall,"Emily Blunt, Lin-Manuel Miranda, Ben Whishaw, ..."
6,A Star Is Born,"[Drama, Music, Romance]",136,215.29,Bradley Cooper,"Lady Gaga, Bradley Cooper, Sam Elliott, Greg G..."


In [18]:
# Transforming the Gross column to a numeric value
df["Gross"] = df["Gross"].str.replace("$","")
df["Gross"] = df["Gross"].str.replace("M","")
df["Gross"] = df["Gross"].str.replace(",","")
df["Gross"] = pd.to_numeric(df["Gross"])



In [19]:
# Same for duration
df["Duration"] = pd.to_numeric(df["Duration"])


In [20]:
df.head()

,Movie_Name,Genres,Duration,Gross,Directors,Stars
0,Bohemian Rhapsody,"[Biography, Drama, Music]",134,216.43,Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy"
1,Spider-Man: Into the Spider-Verse,"[Animation, Action, Adventure]",117,190.24,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Shameik Moore, Jake Johnson, Hailee Steinfeld,..."
3,Hereditary,"[Drama, Horror, Mystery]",127,44.07,Ari Aster,"Toni Collette, Milly Shapiro, Gabriel Byrne, A..."
5,Mary Poppins Returns,"[Comedy, Family, Fantasy]",130,171.96,Rob Marshall,"Emily Blunt, Lin-Manuel Miranda, Ben Whishaw, ..."
6,A Star Is Born,"[Drama, Music, Romance]",136,215.29,Bradley Cooper,"Lady Gaga, Bradley Cooper, Sam Elliott, Greg G..."


In [21]:
df = df.sort_values(by = ["Gross"], axis = 0, ascending = False)
    

In [22]:
df["Gross"].value_counts(normalize = True, bins =2)
#filtered_data = df["Gross"] > 

(-0.701, 350.03]    0.994962
(350.03, 700.06]    0.005038
Name: Gross, dtype: float64

In [23]:
df.shape

(794, 6)

In [24]:
# keeping only blockbusters
new_df = df[df["Gross"] > 100]

In [25]:
type(new_df)

pandas.core.frame.DataFrame

In [26]:
new_df.shape

(33, 6)

In [27]:
new_df

,Movie_Name,Genres,Duration,Gross,Directors,Stars
26,Black Panther,"[Action, Adventure, Sci-Fi]",134,700.06,Ryan Coogler,"Chadwick Boseman, Michael B. Jordan, Lupita Ny..."
7,Avengers: Infinity War,"[Action, Adventure, Sci-Fi]",149,678.82,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Hemsworth, Mark Ruffa..."
84,Incredibles 2,"[Animation, Action, Adventure]",118,608.58,Brad Bird,"Craig T. Nelson, Holly Hunter, Sarah Vowell, H..."
60,Jurassic World: Fallen Kingdom,"[Action, Adventure, Sci-Fi]",128,417.72,J.A. Bayona,"Chris Pratt, Bryce Dallas Howard, Rafe Spall, ..."
11,Aquaman,"[Action, Adventure, Fantasy]",143,335.06,James Wan,"Jason Momoa, Amber Heard, Willem Dafoe, Patric..."
23,Deadpool 2,"[Action, Adventure, Comedy]",119,324.59,David Leitch,"Ryan Reynolds, Josh Brolin, Morena Baccarin, J..."
175,The Grinch,"[Animation, Comedy, Family]",85,270.62,"Yarrow Cheney, Scott Mosier","Benedict Cumberbatch, Cameron Seely, Rashida J..."
39,Mission: Impossible - Fallout,"[Action, Adventure, Thriller]",147,220.16,Christopher McQuarrie,"Tom Cruise, Henry Cavill, Ving Rhames, Simon Pegg"
93,Ant-Man and the Wasp,"[Action, Adventure, Comedy]",118,216.65,Peyton Reed,"Paul Rudd, Evangeline Lilly, Michael Peña, Wal..."
0,Bohemian Rhapsody,"[Biography, Drama, Music]",134,216.43,Bryan Singer,"Rami Malek, Lucy Boynton, Gwilym Lee, Ben Hardy"


## Adding features for Stars 

In [28]:
new_df["Stars"] = new_df["Stars"].str.split(",")
new_df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Movie_Name,Genres,Duration,Gross,Directors,Stars
26,Black Panther,"[Action, Adventure, Sci-Fi]",134,700.06,Ryan Coogler,"[Chadwick Boseman, Michael B. Jordan, Lupita..."
7,Avengers: Infinity War,"[Action, Adventure, Sci-Fi]",149,678.82,"Anthony Russo, Joe Russo","[Robert Downey Jr., Chris Hemsworth, Mark Ru..."
84,Incredibles 2,"[Animation, Action, Adventure]",118,608.58,Brad Bird,"[Craig T. Nelson, Holly Hunter, Sarah Vowell..."
60,Jurassic World: Fallen Kingdom,"[Action, Adventure, Sci-Fi]",128,417.72,J.A. Bayona,"[Chris Pratt, Bryce Dallas Howard, Rafe Spal..."
11,Aquaman,"[Action, Adventure, Fantasy]",143,335.06,James Wan,"[Jason Momoa, Amber Heard, Willem Dafoe, Pa..."


In [38]:
for star in new_df["Stars"]:
    new_df["star_{star}"]= new_df.Stars.apply(lambda star_list: star in star_list)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
new_df

,Movie_Name,Genres,Duration,Gross,Directors,Stars,star_{star}
26,Black Panther,"[Action, Adventure, Sci-Fi]",134,700.06,Ryan Coogler,"[Chadwick Boseman, Michael B. Jordan, Lupita...",False
7,Avengers: Infinity War,"[Action, Adventure, Sci-Fi]",149,678.82,"Anthony Russo, Joe Russo","[Robert Downey Jr., Chris Hemsworth, Mark Ru...",False
84,Incredibles 2,"[Animation, Action, Adventure]",118,608.58,Brad Bird,"[Craig T. Nelson, Holly Hunter, Sarah Vowell...",False
60,Jurassic World: Fallen Kingdom,"[Action, Adventure, Sci-Fi]",128,417.72,J.A. Bayona,"[Chris Pratt, Bryce Dallas Howard, Rafe Spal...",False
11,Aquaman,"[Action, Adventure, Fantasy]",143,335.06,James Wan,"[Jason Momoa, Amber Heard, Willem Dafoe, Pa...",False
23,Deadpool 2,"[Action, Adventure, Comedy]",119,324.59,David Leitch,"[Ryan Reynolds, Josh Brolin, Morena Baccarin...",False
175,The Grinch,"[Animation, Comedy, Family]",85,270.62,"Yarrow Cheney, Scott Mosier","[Benedict Cumberbatch, Cameron Seely, Rashid...",False
39,Mission: Impossible - Fallout,"[Action, Adventure, Thriller]",147,220.16,Christopher McQuarrie,"[Tom Cruise, Henry Cavill, Ving Rhames, Sim...",False
93,Ant-Man and the Wasp,"[Action, Adventure, Comedy]",118,216.65,Peyton Reed,"[Paul Rudd, Evangeline Lilly, Michael Peña, ...",False
0,Bohemian Rhapsody,"[Biography, Drama, Music]",134,216.43,Bryan Singer,"[Rami Malek, Lucy Boynton, Gwilym Lee, Ben ...",False


# Adding New Source for bankable index

In [ ]:
import pandas as pd
bankable = pd.read_csv("/Users/fatimaloumaini/Documents/IMDB_Project/bankable.csv")
bankable = bankable.rename(columns = {"Unnamed: 2":"Actor/Actress","Unnamed: 3": "Total Gross","Unnamed: 4":"Movies/Average", "Unnamed: 5":"Gross/Movie", "Unnamed: 6":"Movie_Name","Unnamed: 7":"Gross_Movie_Name"})
bankable = bankable.drop(["Unnamed: 0","Unnamed: 1"], axis=1)
bankable.drop_duplicates(subset ="Actor/Actress", keep = 'first', inplace = True)


In [ ]:
bankable.head()

In [ ]:
bankable[bankable["Actor/Actress"]=="Emily Blunt"]

In [ ]:
bankable = bankable.sort_values(by = "Total Gross", axis = 0)

In [ ]:
bankable.head()

## To remove

In [ ]:
#df.head()

In [ ]:
# for star in Stars:
#     df[f"genre_{star}"] = df.Genres.apply(lambda star_list: star in star_list)

In [ ]:
#### I am not splitting the directors to 1 to many and not following the same logic for actors

I have to split actors on different categories:
- A list: 2-4 movies
- Supporting: 4-10 movies
- Day Players: 12-15 movies: I shouldn't have much of them on my dataset

In [ ]:
# new_df.columns

In [ ]:
# actors_count = new_df["star_Rami Malek"].value_counts()
# actors_count

In [ ]:
# new_df[new_df["star_Rami Malek"] == True]

In [ ]:
# def split_data_frame_list(dataframe,column,output_type= str):
#     ''' 
#     Accepts a column with multiple types and splits list variables to several rows.

#     df: dataframe to split
#     target_column: the column containing the values to split
#     output_type: type of all outputs
#     returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
#     The values in the other columns are duplicated across the newly divided rows.
#     '''
#     row_accumulator = []

#     def split_list_to_rows(row):
#         split_row = row["Genres"]
#         if isinstance(split_row, list):
            
#             for s in split_row:
#                 new_row = row.to_dict()
#                 new_row["Genres"] = output_type(s)
#                 row_accumulator.append(new_row)
#         else:
            
#             new_row = row.to_dict()
#             new_row[target_column] = output_type(split_row)
#             row_accumulator.append(new_row)
  
#     df.apply(split_list_to_rows, axis=1)
#     new_df = pd.DataFrame(row_accumulator)
  
#     return new_df

In [ ]:
# new_df =split_data_frame_list(df, "Genres",output_type= str)